In [1]:
from bs4 import BeautifulSoup
import csv
import requests
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service  # Import Service
from selenium.webdriver.support import expected_conditions as EC
import json
import datetime
import os
from datetime import datetime
import time
import pandas as pd
import re

# Hard Rock

In [ ]:
### Hard Rock ###

# ChromeDriver path and Chrome options
chromedriver_path = "/opt/homebrew/bin/chromedriver"  # Adjust this path based on where your ChromeDriver is installed
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL to scrape
url = 'https://app.hardrock.bet/home/competition/nfl/2233719185909481580'
driver.get(url)

# Wait for the page to load completely
time.sleep(5)

# Define the tabs to click on, these are found using their data-cy attributes or other identifiers
tabs = {
    "Game Lines": "//div[@data-cy='game-details-tab:Game Lines']",
    "Player Props": "//div[@data-cy='game-details-tab:Player Props']",
    "Halves": "//div[@data-cy='game-details-tab:Halves']",
    "Quarters": "//div[@data-cy='game-details-tab:Quarters']"
}

# Initialize the CSV data
csv_data = []
csv_data.append(["Tab", "Market", "Selection", "Odds"])

# Loop through each tab, simulate the click, and scrape the data
for tab_name, tab_xpath in tabs.items():
    tab_element = driver.find_element(By.XPATH, tab_xpath)
    
    # Use JavaScript to trigger the click
    driver.execute_script("arguments[0].click();", tab_element)
    
    # Wait for the content to load
    time.sleep(3)  # Adjust the time based on the complexity of the page
    
    # Get the page source after clicking and parse it with BeautifulSoup
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Find all 'marketView' sections across the page
    market_views = soup.find_all("div", class_="marketView")

    # Extract and classify data by looking at the current tab section
    for market in market_views:
        # Extract the market header, selection names, and odds
        header = market.find("div", class_="marketHeader").get_text(strip=True) if market.find("div", class_="marketHeader") else "No Header"
        selections = market.find_all("div", class_="selectionName")
        odds = market.find_all("div", class_="selection-odds")
        
        for selection, odd in zip(selections, odds):
            selection_name = selection.get_text(strip=True)
            selection_odds = odd.get_text(strip=True)
            csv_data.append([tab_name, header, selection_name, selection_odds])

# Save the extracted data to a CSV file
csv_path = 'hard_rock_props.csv'
with open(csv_path, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)

print(f"All tabs' data has been saved to {csv_path}")

# Close the browser after scraping
driver.quit()
!open hard_rock_props.csv

# Bovada

In [46]:
### Bovada ###

# ChromeDriver path and Chrome options
chromedriver_path = "/opt/homebrew/bin/chromedriver"  # Adjust this path based on where your ChromeDriver is installed
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL to scrape
url = 'https://www.bovada.lv/sports/football/nfl/new-york-giants-pittsburgh-steelers-202410282015'
driver.get(url)

# Wait for the page to load completely
time.sleep(5)

# Define the tabs to click on, targeting the title attributes of the 'li' elements
tabs = {
    "Alternate Lines": "//li[@title='Alternate Lines']",
    "TD Scorer Props": "//li[@title='TD Scorer Props']",
    "Passing Props": "//li[@title='Passing Props']",
    "Receiving Props": "//li[@title='Receiving Props']",
    "Rushing Props": "//li[@title='Rushing Props']"
}

# Loop through each tab, simulate the click, and save the raw HTML
for tab_name, tab_xpath in tabs.items():
    try:
        # Find the tab element and click on it
        tab_element = driver.find_element(By.XPATH, tab_xpath)
        driver.execute_script("arguments[0].click();", tab_element)
        
        # Wait for the content to load
        time.sleep(3)  # Adjust the time based on the complexity of the page
        
        # Get the page source after clicking
        html_content = driver.page_source
        
        # Save the HTML content to a file
        with open(f"data/bovada/{tab_name.replace(' ', '_')}_tab.html", "w", encoding="utf-8") as file:
            file.write(html_content)
        
        print(f"Saved HTML content for {tab_name} tab.")

    except Exception as e:
        print(f"Error processing tab {tab_name}: {e}")

# Close the browser after scraping
driver.quit()


Saved HTML content for Alternate Lines tab.
Saved HTML content for TD Scorer Props tab.
Saved HTML content for Passing Props tab.
Saved HTML content for Receiving Props tab.
Saved HTML content for Rushing Props tab.


In [10]:
# Receiving Props

def clean_odds(odds_text):
    """Clean odds text to standardized format."""
    if not odds_text:
        return None
    odds_text = odds_text.strip()
    if odds_text == "EVEN":
        return 100
    try:
        return int(re.sub(r'[^0-9-]', '', odds_text))
    except:
        return None

def extract_all_receiving_props(html_path):
    # Read HTML file
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    props_data = []

    # Loop through all headers to capture all occurrences
    headers = soup.find_all('header', {'class': 'game-heading'})

    for header in headers:
        h3_elem = header.find('h3', {'class': 'league-header full-width'})
        if not h3_elem:
            continue

        market_name = h3_elem.text.strip()

        # Find all market containers under the current header
        market_containers = header.find_all_next('section', {'class': 'coupon-content markets-container'})
        
        for market_container in market_containers:
            # Extract player and team
            match = re.search(r'Total (.+?) - (.+?) \((.+?)\)', market_name)
            if not match:
                continue

            prop_type = match.group(1).strip()
            player = match.group(2).strip()
            team = match.group(3).strip()

            # Extract line
            spread_header = market_container.find('ul', {'class': 'spread-header'})
            line = None
            if spread_header and spread_header.find('li'):
                try:
                    line = float(spread_header.find('li').text.strip())
                except ValueError:
                    continue

            # Extract odds
            odds_spans = market_container.find_all('span', {'class': 'bet-price'})
            over_odds = clean_odds(odds_spans[0].text) if len(odds_spans) > 0 else None
            under_odds = clean_odds(odds_spans[1].text) if len(odds_spans) > 1 else None

            props_data.append({
                'Prop_Type': prop_type,
                'Player': player,
                'Team': team,
                'Line': line,
                'Over_Odds': over_odds,
                'Under_Odds': under_odds
            })

    # Convert to DataFrame
    df = pd.DataFrame(props_data)
    return df

def main():
    try:
        # Extract data
        df = extract_all_receiving_props('Receiving_Props_tab.html')
        
        if df.empty:
            print("No props were found in the HTML file.")
            return

        # Save to CSV
        df.to_csv('receiving_props.csv', index=False)
        print(f"Saved {len(df)} props to all_receiving_props.csv")

        # Display first few rows
        print("\nFirst few rows of extracted data:")
        print(df.head())

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e

if __name__ == "__main__":
    main()

Saved 399 props to all_receiving_props.csv

First few rows of extracted data:
         Prop_Type             Player Team  Line  Over_Odds  Under_Odds
0  Receiving Yards  Calvin Austin III  PIT  18.5       -110        -120
1  Receiving Yards  Calvin Austin III  PIT  25.5       -110        -120
2  Receiving Yards  Calvin Austin III  PIT   8.5       -120        -110
3  Receiving Yards  Calvin Austin III  PIT  59.5       -115        -115
4  Receiving Yards  Calvin Austin III  PIT  13.5       -110        -120


In [8]:
# Rushing Props

def clean_odds(odds_text):
    """Clean odds text to standardized format."""
    if not odds_text:
        return None
    odds_text = odds_text.strip()
    if odds_text == "EVEN":
        return 100
    try:
        return int(re.sub(r'[^0-9-]', '', odds_text))
    except:
        return None

def extract_all_rushing_props(html_path):
    # Read HTML file
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
    
    props_data = []
    
    # Find all prop headers using the correct class structure
    headers = soup.find_all('header', {'class': 'game-heading'})
    
    for header in headers:
        # Get market name from h3 element
        h3_elem = header.find('h3', {'class': 'league-header full-width'})
        if not h3_elem:
            continue
            
        market_name = h3_elem.text.strip()
        
        # Find all associated market containers for this header
        market_containers = header.find_all_next('section', {'class': 'coupon-content markets-container'})
        
        for market_container in market_containers:
            # Extract player name and team
            match = re.search(r'Total (.+?) - (.+?) \((.+?)\)', market_name)
            if not match:
                continue
            
            prop_type = match.group(1).strip()
            player = match.group(2).strip()
            team = match.group(3).strip()
            
            # Extract line 
            spread_header = market_container.find('ul', {'class': 'spread-header'})
            line = None
            if spread_header and spread_header.find('li'):
                try:
                    line = float(spread_header.find('li').text.strip())
                except ValueError:
                    continue
            
            # Find odds values - look for bet-price spans
            odds_spans = market_container.find_all('span', {'class': 'bet-price'})
            over_odds = clean_odds(odds_spans[0].text) if len(odds_spans) > 0 else None
            under_odds = clean_odds(odds_spans[1].text) if len(odds_spans) > 1 else None
            
            props_data.append({
                'Prop_Type': prop_type,
                'Player': player,
                'Team': team,
                'Line': line,
                'Over_Odds': over_odds,
                'Under_Odds': under_odds
            })

    # Convert to DataFrame
    df = pd.DataFrame(props_data)
    return df

def main():
    try:
        # Extract data
        df = extract_all_rushing_props('Rushing_Props_tab.html')
        
        if df.empty:
            print("No props were found in the HTML file.")
            return
            
        # Save to CSV
        csv_path = 'rushing_props.csv'
        df.to_csv(csv_path, index=False)
        print(f"Saved {len(df)} props to all_rushing_props.csv")
        
        # Display first few rows
        print("\nFirst few rows of extracted data:")
        print(df.head())
        return csv_path, df.head()
        
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e

if __name__ == "__main__":
    main()


Saved 55 props to all_rushing_props.csv

First few rows of extracted data:
                   Prop_Type         Player Team  Line  Over_Odds  Under_Odds
0  Rushing & Receiving Yards  Jaylen Warren  PIT  47.5       -115        -115
1  Rushing & Receiving Yards  Jaylen Warren  PIT  65.5       -115        -115
2  Rushing & Receiving Yards  Jaylen Warren  PIT  24.5       -115        -115
3  Rushing & Receiving Yards  Jaylen Warren  PIT  31.5       -115        -115
4  Rushing & Receiving Yards  Jaylen Warren  PIT  30.5       -115        -115


In [7]:
# Passing Props

def clean_odds(odds_text):
    """Clean odds text to standardized format."""
    if not odds_text:
        return None
    odds_text = odds_text.strip()
    if odds_text == "EVEN":
        return 100
    try:
        return int(re.sub(r'[^0-9-]', '', odds_text))
    except:
        return None

def extract_all_props(html_path):
    # Read HTML file
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    props_data = []

    # Find all prop headers using the correct class structure
    headers = soup.find_all('header', {'class': 'game-heading'})

    for header in headers:
        # Get market name from h3 element
        h3_elem = header.find('h3', {'class': 'league-header full-width'})
        if not h3_elem:
            continue

        market_name = h3_elem.text.strip()

        # Find all associated market containers for this header
        market_containers = header.find_all_next('section', {'class': 'coupon-content markets-container'})

        for market_container in market_containers:
            # Extract player name and team
            match = re.search(r'Total (.+?) - (.+?) \((.+?)\)', market_name)
            if not match:
                continue

            prop_type = match.group(1).strip()
            player = match.group(2).strip()
            team = match.group(3).strip()

            # Extract line
            spread_header = market_container.find('ul', {'class': 'spread-header'})
            line = None
            if spread_header and spread_header.find('li'):
                try:
                    line = float(spread_header.find('li').text.strip())
                except ValueError:
                    continue

            # Find odds values - look for bet-price spans
            odds_spans = market_container.find_all('span', {'class': 'bet-price'})
            over_odds = clean_odds(odds_spans[0].text) if len(odds_spans) > 0 else None
            under_odds = clean_odds(odds_spans[1].text) if len(odds_spans) > 1 else None

            props_data.append({
                'Prop_Type': prop_type,
                'Player': player,
                'Team': team,
                'Line': line,
                'Over_Odds': over_odds,
                'Under_Odds': under_odds
            })

    # Convert to DataFrame
    df = pd.DataFrame(props_data)
    return df

def main():
    try:
        # Extract data
        df = extract_all_props('Passing_Props_tab.html')
        
        if df.empty:
            print("No props were found in the HTML file.")
            return

        # Save to CSV
        df.to_csv('passing_props.csv', index=False)
        print(f"Saved {len(df)} props to all_props.csv")

        # Display first few rows
        print("\nFirst few rows of extracted data:")
        print(df.head())

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e

if __name__ == "__main__":
    main()


Saved 75 props to all_props.csv

First few rows of extracted data:
       Prop_Type        Player Team   Line  Over_Odds  Under_Odds
0  Passing Yards  Daniel Jones  NYG  187.5       -115        -115
1  Passing Yards  Daniel Jones  NYG  201.5       -115        -115
2  Passing Yards  Daniel Jones  NYG    0.5       -145         110
3  Passing Yards  Daniel Jones  NYG    1.5        155        -210
4  Passing Yards  Daniel Jones  NYG   19.5        105        -135


In [22]:
# Alternate Lines

def clean_odds(odds_text):
    if not odds_text:
        return None
    odds_text = odds_text.strip()
    if odds_text == "EVEN":
        return 100
    try:
        return int(re.sub(r'[^0-9-]', '', odds_text))
    except:
        return None

def extract_all_alternate_lines(html_path):
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    lines_data = []

    headers = soup.find_all('header', {'class': 'game-heading'})

    for header in headers:
        h3_elem = header.find('h3', {'class': 'league-header full-width'})
        if not h3_elem:
            continue

        market_name = h3_elem.text.strip()

        market_containers = header.find_next_siblings('section', {'class': 'coupon-content markets-container'})
        
        for market_container in market_containers:
            spread_header = market_container.find('ul', {'class': 'spread-header'})
            line = None
            if spread_header and spread_header.find('li'):
                try:
                    line = float(spread_header.find('li').text.strip())
                except ValueError:
                    continue

            odds_spans = market_container.find_all('span', {'class': 'bet-price'})
            over_odds = clean_odds(odds_spans[0].text) if len(odds_spans) > 0 else None
            under_odds = clean_odds(odds_spans[1].text) if len(odds_spans) > 1 else None

            lines_data.append({
                'Market_Name': market_name,
                'Line': line,
                'Over_Odds': over_odds,
                'Under_Odds': under_odds
            })

    df = pd.DataFrame(lines_data)
    
    # Save to CSV file
    df.to_csv('alternate_lines.csv', index=False)
    
    return df

def main():
    try:
        df = extract_all_alternate_lines('Alternate_Lines_tab.html')
        
        if df.empty:
            print("No alternate lines were found in the HTML file.")
            return

        print(f"Saved {len(df)} alternate lines to alternate_lines.csv")

        print("\nFirst few rows of extracted data:")
        print(df.head())

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise e

if __name__ == "__main__":
    main()


Saved 12 alternate lines to alternate_lines.csv

First few rows of extracted data:
                      Market_Name  Line  Over_Odds  Under_Odds
0                          Spread   NaN        120         110
1            Spread  - First Half   NaN        130         120
2           Spread  - 1st Quarter   NaN       -135         105
3                    Total Points  34.0       -160        -150
4  Total Points - New York Giants  14.5       -105        -125


In [50]:
# Touchdown Scorer Props

def extract_td_scorer_props(html_path):
    # Read HTML file
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Initialize a list to store the TD Scorer Props data
    td_scorer_props_data_v2 = []

    # Loop through all relevant sections for outcomes and odds
    for outcome in soup.find_all("button", class_="bet-btn"):
        # Extract player name correctly
        player_name = outcome.find("span", class_="outcomes").get_text(strip=True) if outcome.find("span", class_="outcomes") else None
        
        # Extract the odds
        odds = outcome.find("span", class_="bet-price").get_text(strip=True) if outcome.find("span", class_="bet-price") else None

        # Append only if both player name and odds are found
        if player_name and odds:
            td_scorer_props_data_v2.append([player_name, odds])

    # Convert the list to a pandas DataFrame
    df_td_scorer_props_final_v2 = pd.DataFrame(td_scorer_props_data_v2, columns=["Player", "Odds"])

    # Save the DataFrame to a CSV file
    csv_filename = 'data/bovada/td_scorer_props.csv'
    df_td_scorer_props_final_v2.to_csv(csv_filename, index=False)
    print(f"Data saved to {csv_filename}")

    # Display the DataFrame using pandas
    print("Refined TD Scorer Props Data:")
    print(df_td_scorer_props_final_v2)

    # Show the first few rows
    return df_td_scorer_props_final_v2.head()

# Example usage
if __name__ == "__main__":
    df_head = extract_td_scorer_props('data/bovada/TD_Scorer_Props_tab.html')
    print("\nFirst few rows of the DataFrame:")
    print(df_head)

Data saved to data/bovada/td_scorer_props.csv
Refined TD Scorer Props Data:
                       Player    Odds
0         No Touchdown Scorer    -600
1          Malik Nabers (NYG)   +1400
2      Devin Singletary (NYG)   +1800
3      Tyrone Tracy Jr. (NYG)   +1800
4     Wan'dale Robinson (NYG)   +2000
..                        ...     ...
99     Daniel Bellinger (NYG)  +17500
100           Eric Gray (NYG)  +17500
101      Chris Manhertz (NYG)  +20000
102     Rodney Williams (PIT)  +20000
103  Bryce Ford-Wheaton (NYG)  +25000

[104 rows x 2 columns]

First few rows of the DataFrame:
                    Player   Odds
0      No Touchdown Scorer   -600
1       Malik Nabers (NYG)  +1400
2   Devin Singletary (NYG)  +1800
3   Tyrone Tracy Jr. (NYG)  +1800
4  Wan'dale Robinson (NYG)  +2000


In [27]:
# D/ST Props

from bs4 import BeautifulSoup
import pandas as pd

# Function to extract D/ST props
def extract_dst_props(html_path):
    # Read HTML file
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Initialize a list to store the D/ST Props data
    dst_props_data = []

    # Loop through all relevant sections for player names and odds
    for section in soup.find_all("section", class_="coupon-content markets-container"):
        # Find the player name
        player_header = section.find_previous("h3")
        player_name = player_header.get_text(strip=True) if player_header else None

        # Extract over and under values
        spread_header = section.find("ul", class_="spread-header")
        over_value = spread_header.find("li").get_text(strip=True) if spread_header else None

        # Extract the odds for over and under
        market_types = section.find_all("ul", class_="market-type")
        if len(market_types) >= 2:
            over_odds = market_types[0].find("span", class_="bet-price").get_text(strip=True) if market_types[0].find("span", class_="bet-price") else None
            under_odds = market_types[1].find("span", class_="bet-price").get_text(strip=True) if market_types[1].find("span", class_="bet-price") else None

            # Append the data if all values are available
            if player_name and over_value and over_odds and under_odds:
                dst_props_data.append([player_name, over_value, over_odds, under_odds])

    # Convert the list to a pandas DataFrame
    df_dst_props = pd.DataFrame(dst_props_data, columns=["Player", "Over Value", "Over Odds", "Under Odds"])

    # Save the DataFrame to a CSV file
    csv_filename = 'dst_props.csv'
    df_dst_props.to_csv(csv_filename, index=False)
    print(f"Data saved to {csv_filename}")

    # Display the DataFrame using pandas
    print("D/ST Props Data:")
    print(df_dst_props)

    # Show the first few rows
    return df_dst_props.head()

# Example usage
if __name__ == "__main__":
    df_head = extract_dst_props('D_ST_Props_tab.html')
    print("\nFirst few rows of the DataFrame:")
    print(df_head)


Data saved to dst_props.csv
D/ST Props Data:
                                              Player Over Value Over Odds  \
0  Total Tackles and Assists - Cameron Heyward (PIT)        3.5      -115   
1     Total Tackles and Assists - Dexter Lawrence II        3.5      +125   
2    Total Tackles and Assists - Jason Pinnock (NYG)        4.5      -160   
3        Total Tackles and Assists - Joey Porter Jr.        3.5      -160   
4  Total Tackles and Assists - Minkah Fitzpatrick...        6.5      -105   
5    Total Tackles and Assists - Patrick Queen (PIT)        6.5      -145   
6        Total Tackles and Assists - T.J. Watt (PIT)        3.5      -120   
7      Total Tackles and Assists - Tyler Nubin (NYG)        6.5      -135   
8         Total Kicking Points - Chris Boswell (PIT)        6.5      -135   
9            Total Kicking Points - Greg Joseph (GB)        5.5      -115   

  Under Odds  
0       -115  
1       -165  
2       +120  
3       +120  
4       -125  
5       +110  
6 

In [ ]:
# Touchdown Props

from bs4 import BeautifulSoup
import pandas as pd

# Function to extract Touchdown Scorer props
def extract_td_scorer_props(html_path):
    # Read HTML file
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Initialize a list to store the Touchdown Scorer Props data
    td_scorer_props_data = []

    # Loop through all 'sp-outcome' tags that contain the props data
    for outcome in soup.find_all("sp-outcome"):
        # Extract player name from the 'span' with class 'outcomes'
        player_name = outcome.find("span", class_="outcomes").get_text(strip=True) if outcome.find("span", class_="outcomes") else None
        
        # Extract the odds from the 'span' with class 'bet-price'
        td_odds = outcome.find("span", class_="bet-price").get_text(strip=True) if outcome.find("span", class_="bet-price") else None

        # Append only if both player name and odds are found
        if player_name and td_odds:
            td_scorer_props_data.append([player_name, td_odds])

    # Convert the list to a pandas DataFrame
    df_td_scorer_props = pd.DataFrame(td_scorer_props_data, columns=["Player", "TD Scorer Odds"])

    # Save the DataFrame to a CSV file
    csv_filename = 'td_scorer_props.csv'
    df_td_scorer_props.to_csv(csv_filename, index=False)
    print(f"Data saved to {csv_filename}")

    # Display the DataFrame using pandas
    print("Touchdown Scorer Props Data:")
    print(df_td_scorer_props)

    # Show the first few rows
    return df_td_scorer_props.head()

# Example usage
if __name__ == "__main__":
    df_head = extract_td_scorer_props('Touchdown_Props_tab.html')
    print("\nFirst few rows of the DataFrame:")
    print(df_head)


In [ ]:
# Parlays
# Score Props
# Correct Score

In [31]:
# Special Bets

# Function to extract Special Bets
def extract_special_bets(html_path):
    # Read HTML file
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Initialize a list to store the Special Bets data
    special_bets_data = []

    # Loop through all 'sp-outcome' tags that contain special bet data
    for outcome in soup.find_all("sp-outcome"):
        # Extract description of the bet from the 'span' with class 'outcomes'
        bet_description = outcome.find("span", class_="outcomes").get_text(strip=True) if outcome.find("span", class_="outcomes") else None
        
        # Extract the odds from the 'span' with class 'bet-price'
        bet_odds = outcome.find("span", class_="bet-price").get_text(strip=True) if outcome.find("span", class_="bet-price") else None

        # Append the data if both bet description and odds are found
        if bet_description and bet_odds:
            special_bets_data.append([bet_description, bet_odds])

    # Convert the list to a pandas DataFrame
    df_special_bets = pd.DataFrame(special_bets_data, columns=["Bet Description", "Odds"])

    # Save the DataFrame to a CSV file
    csv_filename = 'special_bets.csv'
    df_special_bets.to_csv(csv_filename, index=False)
    print(f"Data saved to {csv_filename}")

    # Display the DataFrame using pandas
    print("Special Bets Data:")
    print(df_special_bets)

    # Show the first few rows
    return df_special_bets.head()

# Example usage
if __name__ == "__main__":
    df_head = extract_special_bets('Special_Bets_tab.html')
    print("\nFirst few rows of the DataFrame:")
    print(df_head)


Data saved to special_bets.csv
Special Bets Data:
                                      Bet Description    Odds
0   Najee Harris to record 25+ Rushing Yards in Ea...    +140
1   Malik Nabers to record 25+ Receiving Yards in ...    +150
2                                  u30.5 Total Points    +200
3   George Pickens to record 25+ Receiving Yards i...    +220
4              Both teams to score 1+ TD in each half    +390
5   George Pickens and Malik Nabers to Each have 2...    +400
6                     Russell Wilson 2+ Interceptions    +475
7                                  u23.5 Total Points    +600
8   Each team to score 1+ rushing TDs & 1+ passing...    +625
9   George Pickens & Malik Nabers to Each Record 5...   +1500
10  D.Jones 192+ Pass Yds, R.Wilson 202+ Pass Yds,...   +1800
11  Russell Wilson & Daniel Jones 2+ Interceptions...   +3000
12      Each team to score 1+ TD & 1+ FG in each half   +3300
13  W.Robinson 50+ Receiving Yards, G.Pickens 75+ ...   +4000
14             Both 

In [32]:
# Game Props

# Function to extract Game Props
def extract_game_props(html_path):
    # Read HTML file
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Initialize a list to store the Game Props data
    game_props_data = []

    # Loop through all 'sp-outcome' tags that contain game prop data
    for outcome in soup.find_all("sp-outcome"):
        # Extract description of the prop from the 'span' with class 'outcomes'
        prop_description = outcome.find("span", class_="outcomes").get_text(strip=True) if outcome.find("span", class_="outcomes") else None
        
        # Extract the odds from the 'span' with class 'bet-price'
        prop_odds = outcome.find("span", class_="bet-price").get_text(strip=True) if outcome.find("span", class_="bet-price") else None

        # Append the data if both prop description and odds are found
        if prop_description and prop_odds:
            game_props_data.append([prop_description, prop_odds])

    # Convert the list to a pandas DataFrame
    df_game_props = pd.DataFrame(game_props_data, columns=["Prop Description", "Odds"])

    # Save the DataFrame to a CSV file
    csv_filename = 'game_props.csv'
    df_game_props.to_csv(csv_filename, index=False)
    print(f"Data saved to {csv_filename}")

    # Display the DataFrame using pandas
    print("Game Props Data:")
    print(df_game_props)

    # Show the first few rows
    return df_game_props.head()

# Example usage
if __name__ == "__main__":
    df_head = extract_game_props('Game_Props_tab.html')
    print("\nFirst few rows of the DataFrame:")
    print(df_head)


Data saved to game_props.csv
Game Props Data:
            Prop Description   Odds
0       Giants by 1-6 points   +385
1     Steelers by 1-6 points   +260
2      Giants by 7-12 points   +700
3    Steelers by 7-12 points   +320
4     Giants by 13-18 points  +1200
..                       ...    ...
96                  Odd - 1H   -115
97                 Even - 1Q   -110
98                  Odd - 1Q   -120
99                 Yes - REG   +900
100                 No - REG  -3300

[101 rows x 2 columns]

First few rows of the DataFrame:
          Prop Description   Odds
0     Giants by 1-6 points   +385
1   Steelers by 1-6 points   +260
2    Giants by 7-12 points   +700
3  Steelers by 7-12 points   +320
4   Giants by 13-18 points  +1200


In [38]:
# Score Props

# Now let's create and run two scripts for the Score Props and Parlays, using the structure identified from the previous scripts.

# Function to extract Score Props
def extract_score_props(html_path):
    # Read HTML file
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Initialize a list to store the Score Props data
    score_props_data = []

    # Loop through all 'sp-outcome' tags that contain score prop data
    for outcome in soup.find_all("sp-outcome"):
        # Extract description of the prop from the 'span' with class 'outcomes'
        prop_description = outcome.find("span", class_="outcomes").get_text(strip=True) if outcome.find("span", class_="outcomes") else None
        
        # Extract the odds from the 'span' with class 'bet-price'
        prop_odds = outcome.find("span", class_="bet-price").get_text(strip=True) if outcome.find("span", class_="bet-price") else None

        # Append the data if both prop description and odds are found
        if prop_description and prop_odds:
            score_props_data.append([prop_description, prop_odds])

    # Convert the list to a pandas DataFrame
    df_score_props = pd.DataFrame(score_props_data, columns=["Prop Description", "Odds"])

    # Save the DataFrame to a CSV file
    csv_filename = 'score_props.csv'
    df_score_props.to_csv(csv_filename, index=False)
    print(f"Data saved to {csv_filename}")

    # Return the DataFrame to show the first few rows
    return df_score_props.head()

# Running the function for Score Props
df_head_score_props = extract_score_props('Score_Props_tab.html')
df_head_score_props


Data saved to score_props.csv


,Prop Description,Odds
0,New York Giants,+130
1,Pittsburgh Steelers,-170
2,Yes - 1H,-475
3,No - 1H,+320
4,Yes - 1Q,+235


In [40]:
# Parlays

# Function to extract Parlays
def extract_parlays(html_path):
    # Read HTML file
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Initialize a list to store the Parlays data
    parlays_data = []

    # Loop through all 'sp-outcome' tags that contain parlay data
    for outcome in soup.find_all("sp-outcome"):
        # Extract description of the parlay from the 'span' with class 'outcomes'
        parlay_description = outcome.find("span", class_="outcomes").get_text(strip=True) if outcome.find("span", class_="outcomes") else None
        
        # Extract the odds from the 'span' with class 'bet-price'
        parlay_odds = outcome.find("span", class_="bet-price").get_text(strip=True) if outcome.find("span", class_="bet-price") else None

        # Append the data if both parlay description and odds are found
        if parlay_description and parlay_odds:
            parlays_data.append([parlay_description, parlay_odds])

    # Convert the list to a pandas DataFrame
    df_parlays = pd.DataFrame(parlays_data, columns=["Parlay Description", "Odds"])

    # Save the DataFrame to a CSV file
    csv_filename = 'parlays.csv'
    df_parlays.to_csv(csv_filename, index=False)
    print(f"Data saved to {csv_filename}")

    # Return the DataFrame to show the first few rows
    return df_parlays.head()

# Running the function for Parlays
df_head_parlays = extract_parlays('Parlays_tab.html')
df_head_parlays


,Parlay Description,Odds
0,PIT Steelers Punt - NY Giants Punt,+165
1,PIT Steelers Touchdown - NY Giants Punt,+700
2,PIT Steelers Field Goal Attempt - NY Giants Punt,+800
3,PIT Steelers Punt - NY Giants Field Goal Attempt,+850
4,PIT Steelers Punt - NY Giants Touchdown,+900


In [43]:
# Correct Score

import os
import pandas as pd
from bs4 import BeautifulSoup

# Function to extract and save Correct Score Props to CSV
def extract_correct_score_props_to_csv(html_path, output_csv):
    # Read HTML file
    with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')

    # Initialize a list to store the Correct Score Props data
    correct_score_data = []

    # Loop through all 'sp-outcome' tags that contain correct score data
    for outcome in soup.find_all("sp-outcome"):
        # Extract description of the prop from the 'span' with class 'outcomes'
        score_description = outcome.find("span", class_="outcomes").get_text(strip=True) if outcome.find("span", class_="outcomes") else None
        
        # Extract the odds from the 'span' with class 'bet-price'
        score_odds = outcome.find("span", class_="bet-price").get_text(strip=True) if outcome.find("span", class_="bet-price") else None

        # Append the data if both description and odds are found
        if score_description and score_odds:
            correct_score_data.append([score_description, score_odds])

    # Convert the list to a pandas DataFrame
    df_correct_score = pd.DataFrame(correct_score_data, columns=["Score Description", "Odds"])

    # Check if a directory is specified, and create it if necessary
    output_dir = os.path.dirname(output_csv)
    if output_dir:  # Only attempt to create a directory if a directory is specified
        os.makedirs(output_dir, exist_ok=True)

    # Save the DataFrame to a CSV file
    df_correct_score.to_csv(output_csv, index=False)
    print(f"Data saved to {output_csv}")

    # Return the first few rows of the DataFrame
    return df_correct_score.head()

# Example usage
correct_score_csv_path = 'correct_score.csv'  # or use 'data/correct_score.csv' to specify a directory
extract_correct_score_props_to_csv('Correct_Score_tab.html', correct_score_csv_path)

# df_head_correct_score  # Show the first few rows of the DataFrame


Data saved to correct_score.csv


,Score Description,Odds
0,Steelers 15 or less and Giants between 16 and 22,+750
1,Steelers 15 or less and Giants 15 or less,+550
2,Steelers between 16 and 22 and Giants 15 or less,+400
3,Steelers 15 or less and Giants 23 or more,+900
4,Steelers between 16 and 22 and Giants between ...,+725


# Stake

In [ ]:
/html/body/div[1]/div[2]/div[3]/div[3]/div/div[1]/div/div/div[2]/div/div[1]/div/div/div/div/div[2]/div/div/div/div/div/div[4]
/html/body/div[1]/div[2]/div[3]/div[3]/div/div[1]/div/div/div[2]/div/div[1]/div/div/div/div/div[2]/div/div/div/div/div/div[4]/div[3]
/html/body/div[1]/div[2]/div[3]/div[3]/div/div[1]/div/div/div[2]/div/div[1]/div/div/div/div/div[2]/div/div/div/div/div/div[4]/div[3]/div
/html/body/div[1]/div[2]/div[3]/div[3]/div/div[1]/div/div/div[2]/div/div[1]/div/div/div/div/div[2]/div/div/div/div/div/div[4]/div[1]/div/div/button[1] # Main button

# from bs4 import BeautifulSoup
# import pandas as pd

# # Load the local HTML file
# html_file_path = '/mnt/data/Pittsburgh Steelers VS New York Giants - Stake.com.html'

# with open(html_file_path, 'r', encoding='utf-8') as file:
#     html_content = file.read()

# # Parse the HTML content with BeautifulSoup
# soup = BeautifulSoup(html_content, 'html.parser')

# # Navigating through layers in the HTML as per the user's instructions
# content_wrapper = soup.find('div', class_='content-wrapper')

# if content_wrapper:
#     # Dive into the nested structure for 'page-content' -> 'layout-spacing' -> 'secondary-accordion'
#     page_content = content_wrapper.find('div', id='main-content')
#     layout_spacing = page_content.find('div', class_='layout-spacing') if page_content else None
#     accordions = layout_spacing.find_all('div', class_='secondary-accordion') if layout_spacing else []

#     # Collect all text data from the accordions
#     accordion_data = [accordion.text.strip() for accordion in accordions]

#     # Convert to DataFrame
#     df = pd.DataFrame(accordion_data, columns=['Accordion Content'])

#     # Display the first few rows
#     df.head()

# else:
#     accordion_data = []
#     df = pd.DataFrame()

# # Show the extracted data and return the file path if saving is needed
# df.head(), accordion_data


In [23]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# ChromeDriver path and Chrome options
chromedriver_path = "/opt/homebrew/bin/chromedriver"  # Adjust this path based on where your ChromeDriver is installed
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Load the webpage
url = "https://stake.com/sports/american-football/usa/nfl/44613931-pittsburgh-steelers-new-york-giants"
driver.get(url)

# Give the page some time to fully load (increase if necessary)
time.sleep(5)

# Get the raw HTML of the page
html_content = driver.page_source

# Save the raw HTML to a file
with open("raw_page.html", "w", encoding="utf-8") as file:
    file.write(html_content)

print("Raw HTML saved as 'raw_page.html'")

# Close the browser
driver.quit()


Raw HTML saved as 'raw_page.html'


In [5]:

from bs4 import BeautifulSoup
import pandas as pd

# Load the local HTML file
html_file_path = 'raw_page2.html'
# html_file_path = 'Pittsburgh Steelers VS New York Giants - Stake.com.html'

with open(html_file_path, 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, 'html.parser')

# Navigating through layers in the HTML as per the user's instructions
content_wrapper = soup.find('div', class_='content-wrapper')

if content_wrapper:
    # Dive into the nested structure for 'page-content' -> 'layout-spacing' -> 'secondary-accordion'
    page_content = content_wrapper.find('div', id='main-content')
    layout_spacing = page_content.find('div', class_='layout-spacing') if page_content else None
    accordions = layout_spacing.find_all('div', class_='secondary-accordion') if layout_spacing else []

    # Collect all text data from the accordions
    accordion_data = [accordion.text.strip() for accordion in accordions]

    # Convert to DataFrame
    df = pd.DataFrame(accordion_data, columns=['Accordion Content'])

    # Display the first few rows
    df.head()

else:
    accordion_data = []
    df = pd.DataFrame()

# Show the extracted data and return the file path if saving is needed
df.head(), accordion_data


(Empty DataFrame
 Columns: []
 Index: [],
 [])

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium_stealth import stealth
import random

# Step 1: Set up user-agent rotation
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.1.2 Safari/605.1.15",
    "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:89.0) Gecko/20100101 Firefox/89.0"
    # Add more user agents here
]
user_agent = random.choice(user_agents)

# Step 2: Set up Chrome options
options = webdriver.ChromeOptions()
options.add_argument(f"user-agent={user_agent}")

# Optional: Use a proxy (residential proxies are recommended for bypassing anti-bot systems)
# proxy = "user:pass@ip:port"  # Replace with your proxy details
# options.add_argument(f'--proxy-server={proxy}')

# Initialize the WebDriver with the desired options
driver = webdriver.Chrome(options=options)

# Step 3: Apply Selenium-stealth settings
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",  # Use 'Win32' to emulate a Windows platform
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True,  # Fix for detecting thin hairline differences
        )

# Step 4: Navigate to the page you want to scrape
url = "https://stake.com/sports/american-football/usa/nfl/44613931-pittsburgh-steelers-new-york-giants"
driver.get(url)

# Step 5: Add a delay to allow the page to load (adjust this if necessary)
# driver.implicitly_wait(10)
time.sleep(10)


# Step 6: Scrape the content after Cloudflare challenges
html_content = driver.page_source

# Save the raw HTML to a file
with open("raw_page.html", "w", encoding="utf-8") as file:
    file.write(html_content)

print("Raw HTML saved as 'raw_page.html'")

# Step 7: Close the browser
driver.quit()


NameError: name 'time' is not defined

In [11]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

# Step 1: Set up Chrome options
options = webdriver.ChromeOptions()

# Set up ScrapeOps Proxy with API key
scrapeops_proxy = "proxy.scrapeops.io:5353"
api_key = "08d19650-4b8c-4d3b-83bc-dad61acc9308"
options.add_argument(f'--proxy-server=http://{api_key}:{scrapeops_proxy}')

# Optional: Set up user-agent rotation (you can remove or modify this if not needed)
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")

# Initialize the WebDriver with the desired options
chromedriver_path = "/opt/homebrew/bin/chromedriver"  # Adjust this path based on where your ChromeDriver is installed
service = Service(chromedriver_path)
options.add_argument("--headless")  # Run Chrome in headless mode
driver = webdriver.Chrome(service=service, options=options)

# Step 2: Navigate to the page you want to scrape
url = "https://stake.com/sports/american-football/usa/nfl/44613931-pittsburgh-steelers-new-york-giants"
driver.get(url)

# Step 3: Add a delay to allow the page to load (adjust this if necessary)
time.sleep(5)  # Adjust this time if necessary

# Step 4: Find the first button and click it
try:
    first_button = driver.find_element(By.XPATH, "/html/body/div[1]/div[2]/div[3]/div[3]/div/div[1]/div/div/div[2]/div/div[1]/div/div/div/div/div[2]/div/div/div/div/div/div[4]/div[1]/div/div/button[1]")  # Main button
    first_button.click()
    print("Clicked the first button.")
except Exception as e:
    print(f"Error clicking the button: {e}")

# Step 5: Wait for the page to load after the click
time.sleep(5)  # Adjust this time if necessary

# Step 6: Get the raw HTML of the page after navigating/clicking
html_content = driver.page_source

# Step 5: Save the raw HTML to a file
with open("raw_page_with_scrapeops.html", "w", encoding="utf-8") as file:
    file.write(html_content)

print("Raw HTML saved as 'raw_page_with_scrapeops.html'")

# Step 6: Close the browser
driver.quit()


Error clicking the button: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[1]/div[2]/div[3]/div[3]/div/div[1]/div/div/div[2]/div/div[1]/div/div/div/div/div[2]/div/div/div/div/div/div[4]/div[1]/div/div/button[1]"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x000000010503b9d4 cxxbridge1$str$ptr + 3647524
1   chromedriver                        0x0000000105034234 cxxbridge1$str$ptr + 3616900
2   chromedriver                        0x0000000104aa010c cxxbridge1$string$len + 88416
3   chromedriver                        0x0000000104ae2338 cxxbridge1$string$len + 359308
4   chromedriver                        0x0000000104b1bb10 cxxbridge1$string$len + 594788
5   chromedriver                        0x0000000104ad6f34 cxxbridge1$string$len + 313224
6   ch

In [13]:
import requests
from bs4 import BeautifulSoup

# URL to scrape
url = "https://stake.com/sports/american-football/usa/nfl/44613931-pittsburgh-steelers-new-york-giants"

# Fetch the page content
response = requests.get(url)
html_content = response.text

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find all tables in the HTML
tables = soup.find_all("table")

# Iterate over each table and print its prettified content
for index, table in enumerate(tables):
    print(f"Table {index + 1}:")
    print(table.prettify())
    print("\n" + "="*50 + "\n")

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Step 1: Set up Chrome options
options = webdriver.ChromeOptions()

# Optional: Add a user-agent (you can remove or modify this if not needed)
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")

# Initialize the WebDriver with the desired options
chromedriver_path = "/opt/homebrew/bin/chromedriver"  # Adjust this path based on where your ChromeDriver is installed
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

# Step 2: Navigate to the page you want to scrape
url = "https://stake.com/sports/american-football/usa/nfl/44613931-pittsburgh-steelers-new-york-giants"
driver.get(url)

# Step 3: Add a delay to allow the page to load (adjust this if necessary)
driver.implicitly_wait(10)

# Step 4: Find the "Main" button using its class and click it
try:
    # Use XPath to target the button with the exact text "Main"
    main_button = driver.find_element(By.XPATH, "//button[span[text()='Main']]")
    main_button.click()
    print("Clicked the 'Main' button.")
except Exception as e:
    print(f"Error clicking the button: {e}")

# Step 5: Wait for the page to load after the click
time.sleep(3)  # Adjust this time if necessary

# Step 6: Get the raw HTML of the page after navigating/clicking
html_content = driver.page_source

# Step 7: Save the raw HTML to a file
with open("raw_page_after_click.html", "w", encoding="utf-8") as file:
    file.write(html_content)

print("Raw HTML saved as 'raw_page_after_click.html'")

# Step 8: Close the browser
driver.quit()


Error clicking the button: 'NoneType' object has no attribute 'click'


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.70)
Stacktrace:
0   chromedriver                        0x000000010545b9d4 cxxbridge1$str$ptr + 3647524
1   chromedriver                        0x0000000105454234 cxxbridge1$str$ptr + 3616900
2   chromedriver                        0x0000000104ec010c cxxbridge1$string$len + 88416
3   chromedriver                        0x0000000104e9bb7c core::str::slice_error_fail::h1cab30ac4b13c655 + 3792
4   chromedriver                        0x0000000104f285ac cxxbridge1$string$len + 515584
5   chromedriver                        0x0000000104f3b578 cxxbridge1$string$len + 593356
6   chromedriver                        0x0000000104ef6f34 cxxbridge1$string$len + 313224
7   chromedriver                        0x0000000104ef7ba4 cxxbridge1$string$len + 316408
8   chromedriver                        0x000000010542661c cxxbridge1$str$ptr + 3429484
9   chr

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Step 1: Set up Chrome options
options = webdriver.ChromeOptions()

# Optional: Add a user-agent (you can remove or modify this if not needed)
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")

# Make Chrome run in headless mode
options.add_argument("--headless")

# Step 2: Set up ScrapeOps Proxy with your API key
api_key = '08d19650-4b8c-4d3b-83bc-dad61acc9308'  # Replace with your actual ScrapeOps API key
scrapeops_proxy = f"http://{api_key}:@proxy.scrapeops.io:5353"
options.add_argument(f'--proxy-server={scrapeops_proxy}')

# Initialize the WebDriver with the desired options
chromedriver_path = "/opt/homebrew/bin/chromedriver"  # Adjust this path based on where your ChromeDriver is installed
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

# Step 3: Navigate to the page you want to scrape
url = "https://stake.com/sports/american-football/usa/nfl/44613931-pittsburgh-steelers-new-york-giants"
driver.get(url)

# Step 4: Add a delay to allow the page to load (adjust this if necessary)
driver.implicitly_wait(10)

# Step 5: Find the "Main" button using its class and click it
try:
    # Use XPath to target the button with the exact text "Main"
    main_button = driver.find_element(By.XPATH, "//button[span[text()='Main']]")
    main_button.click()
    print("Clicked the 'Main' button.")
except Exception as e:
    print(f"Error clicking the button: {e}")

# Step 6: Wait for the page to load after the click
time.sleep(3)  # Adjust this time if necessary

# Step 7: Get the raw HTML of the page after navigating/clicking
html_content = driver.page_source

# Step 8: Save the raw HTML to a file
with open("raw_page_after_click.html", "w", encoding="utf-8") as file:
    file.write(html_content)

print("Raw HTML saved as 'raw_page_after_click.html'")

# Step 9: Close the browser
driver.quit()


Error clicking the button: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[span[text()='Main']]"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001030f79d4 cxxbridge1$str$ptr + 3647524
1   chromedriver                        0x00000001030f0234 cxxbridge1$str$ptr + 3616900
2   chromedriver                        0x0000000102b5c10c cxxbridge1$string$len + 88416
3   chromedriver                        0x0000000102b9e338 cxxbridge1$string$len + 359308
4   chromedriver                        0x0000000102bd7b10 cxxbridge1$string$len + 594788
5   chromedriver                        0x0000000102b92f34 cxxbridge1$string$len + 313224
6   chromedriver                        0x0000000102b93ba4 cxxbridge1$string$len + 316408
7   chromedriver                     

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# ChromeDriver path and Chrome options
chromedriver_path = "/opt/homebrew/bin/chromedriver"  # Adjust this path based on where your ChromeDriver is installed
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# URL to scrape
url = "https://stake.com/sports/american-football/usa/nfl/44613931-pittsburgh-steelers-new-york-giants"
driver.get(url)

# Allow the page to load
driver.implicitly_wait(10)

# Get the page source
html_content = driver.page_source

# Parse the HTML content with BeautifulSoup
soup = BeautifulSoup(html_content, "html.parser")

# Find all tables in the HTML
tables = soup.find_all("table")

# Iterate over each table and print its prettified content
for index, table in enumerate(tables):
    print(f"Table {index + 1}:")
    print(table.prettify())
    print("\n" + "="*50 + "\n")

# Close the browser
driver.quit()

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

# Step 1: Set up Chrome options
options = webdriver.ChromeOptions()

# Optional: Add a user-agent (you can remove or modify this if not needed)
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
options.add_argument(f"user-agent={user_agent}")

# Initialize the WebDriver with the desired options
chromedriver_path = "/opt/homebrew/bin/chromedriver"  # Adjust this path based on where your ChromeDriver is installed
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=options)

# Step 2: Navigate to the page you want to scrape
url = "https://stake.com/sports/american-football/usa/nfl/44613931-pittsburgh-steelers-new-york-giants"
driver.get(url)

# Step 3: Add a delay to allow the page to load (adjust this if necessary)
driver.implicitly_wait(10)

# Step 4: Find the first button and click it
try:
    first_button = driver.find_element(By.XPATH, "//button[contains(., 'Main')]")
    first_button.click()
    print("Clicked the first button.")
except Exception as e:
    print(f"Error clicking the button: {e}")

# Step 5: Wait for the page to load after the click
time.sleep(3)  # Adjust this time if necessary

# Step 6: Get the raw HTML of the page after navigating/clicking
html_content = driver.page_source

# Step 7: Save the raw HTML to a file
with open("raw_page_after_click.html", "w", encoding="utf-8") as file:
    file.write(html_content)

print("Raw HTML saved as 'raw_page_after_click.html'")

# Step 8: Close the browser
driver.quit()


Error clicking the button: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[contains(., 'Main')]"}
  (Session info: chrome=130.0.6723.70); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000100ab39d4 cxxbridge1$str$ptr + 3647524
1   chromedriver                        0x0000000100aac234 cxxbridge1$str$ptr + 3616900
2   chromedriver                        0x000000010051810c cxxbridge1$string$len + 88416
3   chromedriver                        0x000000010055a338 cxxbridge1$string$len + 359308
4   chromedriver                        0x0000000100593b10 cxxbridge1$string$len + 594788
5   chromedriver                        0x000000010054ef34 cxxbridge1$string$len + 313224
6   chromedriver                        0x000000010054fba4 cxxbridge1$string$len + 316408
7   chromedriver                     

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=130.0.6723.70)
Stacktrace:
0   chromedriver                        0x0000000100ab39d4 cxxbridge1$str$ptr + 3647524
1   chromedriver                        0x0000000100aac234 cxxbridge1$str$ptr + 3616900
2   chromedriver                        0x000000010051810c cxxbridge1$string$len + 88416
3   chromedriver                        0x00000001004f3b7c core::str::slice_error_fail::h1cab30ac4b13c655 + 3792
4   chromedriver                        0x00000001005805ac cxxbridge1$string$len + 515584
5   chromedriver                        0x0000000100593578 cxxbridge1$string$len + 593356
6   chromedriver                        0x000000010054ef34 cxxbridge1$string$len + 313224
7   chromedriver                        0x000000010054fba4 cxxbridge1$string$len + 316408
8   chromedriver                        0x0000000100a7e61c cxxbridge1$str$ptr + 3429484
9   chr

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

# ChromeDriver path and Chrome options
chromedriver_path = "/opt/homebrew/bin/chromedriver"  # Adjust this path based on where your ChromeDriver is installed
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Open the webpage
url = "https://stake.com/sports/american-football/usa/nfl/44613931-pittsburgh-steelers-new-york-giants"
driver.get(url)

# Allow the page to load
driver.implicitly_wait(10)

# XPath values as provided
xpaths = [
    "/html/body/div[1]/div[2]/div[3]/div[3]/div/div[1]/div/div/div[2]/div/div[1]/div/div/div/div/div[2]/div/div/div/div/div/div[4]",
    "/html/body/div[1]/div[2]/div[3]/div[3]/div/div[1]/div/div/div[2]/div/div[1]/div/div/div/div/div[2]/div/div/div/div/div/div[4]/div[3]",
    "/html/body/div[1]/div[2]/div[3]/div[3]/div/div[1]/div/div/div[2]/div/div[1]/div/div/div/div/div[2]/div/div/div/div/div/div[4]/div[3]/div"
]

# Loop through each XPath and extract the text
betting_data = []
for xpath in xpaths:
    try:
        element = driver.find_element(By.XPATH, xpath)
        betting_data.append(element.text)
    except Exception as e:
        print(f"Could not find element for {xpath}: {e}")

# Convert extracted data into a DataFrame
df = pd.DataFrame(betting_data, columns=['Betting Odds'])
print(df)

# Optionally, save to CSV
df.to_csv("specific_xpath_betting_odds.csv", index=False)

# Close the browser
driver.quit()


NameError: name 'Options' is not defined

In [16]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# ChromeDriver path and Chrome options
chromedriver_path = "/opt/homebrew/bin/chromedriver"  # Adjust this path based on where your ChromeDriver is installed
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Load your local HTML file or the URL
url = "https://stake.com/sports/american-football/usa/nfl/44613931-pittsburgh-steelers-new-york-giants"
driver.get(url)

# Allow the page to load
wait = WebDriverWait(driver, 20)  # Explicit wait up to 20 seconds

# Find the main content by following the path step by step using class names
try:
    # Step 1: Locate the "content-wrapper" div
    content_wrapper = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-wrapper")))
    
    # Step 2: Inside content-wrapper, locate the "page-content"
    page_content = content_wrapper.find_element(By.ID, "main-content")
    
    # Step 3: Inside "page-content", locate the layout spacing container
    layout_spacing = page_content.find_element(By.CLASS_NAME, "layout-spacing")
    
    # Step 4: Locate all "secondary-accordion" elements within this section
    accordions = layout_spacing.find_elements(By.CLASS_NAME, "secondary-accordion")
    
    # Collect all text data from the accordions
    accordion_data = [accordion.text for accordion in accordions]
    
    # Print the data
    for index, data in enumerate(accordion_data):
        print(f"Accordion {index + 1}: {data}")

    # Optionally save to CSV
    df = pd.DataFrame(accordion_data, columns=['Accordion Content'])
    df.to_csv("accordion_data.csv", index=False)
    print("CSV file saved as 'accordion_data.csv'")

except Exception as e:
    print(f"Error encountered: {e}")

# Close the browser
driver.quit()


Error encountered: Message: 
Stacktrace:
0   chromedriver                        0x00000001006c39d4 cxxbridge1$str$ptr + 3647524
1   chromedriver                        0x00000001006bc234 cxxbridge1$str$ptr + 3616900
2   chromedriver                        0x000000010012810c cxxbridge1$string$len + 88416
3   chromedriver                        0x000000010016a338 cxxbridge1$string$len + 359308
4   chromedriver                        0x00000001001a3b10 cxxbridge1$string$len + 594788
5   chromedriver                        0x000000010015ef34 cxxbridge1$string$len + 313224
6   chromedriver                        0x000000010015fba4 cxxbridge1$string$len + 316408
7   chromedriver                        0x000000010068e61c cxxbridge1$str$ptr + 3429484
8   chromedriver                        0x0000000100691958 cxxbridge1$str$ptr + 3442600
9   chromedriver                        0x0000000100675344 cxxbridge1$str$ptr + 3326356
10  chromedriver                        0x000000010069221c cxxbridge1$

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

# ChromeDriver path and Chrome options
chromedriver_path = "/opt/homebrew/bin/chromedriver"  # Adjust this path based on where your ChromeDriver is installed
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run Chrome in headless mode
service = Service(chromedriver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Load your local HTML file
local_file_path = "Scraping/Pittsburgh Steelers VS New York Giants - Stake.com.html"
driver.get(f"file://{local_file_path}")

# Allow the page to load
wait = WebDriverWait(driver, 10)  # Explicit wait up to 20 seconds

# Find the main content by following the path step by step using class names
try:
    # Step 1: Locate the "content-wrapper" div
    content_wrapper = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "content-wrapper")))
    
    # Step 2: Inside content-wrapper, locate the "page-content"
    page_content = content_wrapper.find_element(By.ID, "main-content")
    
    # Step 3: Inside "page-content", locate the layout spacing container
    layout_spacing = page_content.find_element(By.CLASS_NAME, "layout-spacing")
    
    # Step 4: Locate all "secondary-accordion" elements within this section
    accordions = layout_spacing.find_elements(By.CLASS_NAME, "secondary-accordion")
    
    # Collect all text data from the accordions
    accordion_data = [accordion.text for accordion in accordions]
    
    # Print the data
    for index, data in enumerate(accordion_data):
        print(f"Accordion {index + 1}: {data}")

    # Optionally save to CSV
    df = pd.DataFrame(accordion_data, columns=['Accordion Content'])
    df.to_csv("accordion_data.csv", index=False)
    print("CSV file saved as 'accordion_data.csv'")

except Exception as e:
    print(f"Error encountered: {e}")

# Close the browser
driver.quit()
